In [1]:
import pandas as pd
import numpy as np

telecom_df = pd.read_excel('CSI_data_set.xlsx', sheet_name='Data set')

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Проект Телекома

## Постановка задачи и описание данных

Таблица telecom_df - содержит в себе анонимные данные о пользователях услуг сотовой связи в США.

- `state_type` — Тип штата;
- `state` — Штат;
- `city` — Город;
- `segment_gender` — Пол - (1 - ж; 0 - м);
- `age` — Возвраст;
- `age_gr_1` — Сегментация по возрасту;
- `age_gr_2` — Сегментация по возрасту;
- `arpu_segment` — Сегмент по ARPU - (ARPU - average revenue per user);
- `bucket_tr` — Сегмент по объему скаченного трафика за месяц, в Mb;
- `mb_2g_flg` *—* Флаг - наличие трафика в технологии 2G - (Выпадание в 2G - негативное событие для клиента);
- `os_name` *—* Тип операционной системы;
- `cpe_type_name` *—* Тип устройства;
- `lt_day` *—* Срок жизни в днях - (lt - lifetime);
- `base_type` *—* Старый (+12 мес) или новый (менее 12 мес) абонент - (1 - new, 0 - old);
- `lt_gr_1` *—* Сегментация по сроку жизни.
- `lt_gr_2` *—* Сегментация по сроку жизни.
- `csi` *—* Customer satifaction index.
- `csi_category` *—* Категория CSI - (1 - очень несчастный, 2 - очень счастливый).
- `csi_key` *—* Ключ CSI - (1 - happy, 0 - neutral, -1 - unhappy).

В рамках проекта нам нужно:
- проверить данные и указать на их проблемы.
- проверить дубликаты, пропуски и выбросы в данных.
- ответить на поставленные вопросы заказчика.

## Предобработка  и Исследователький анализ данных

In [2]:
telecom_df.sample(5)

,state_type,state,city,segment_gender,age,age_gr_1,age_gr_2,arpu_segment,bucket_tr,mb_2g_flg,os_name,cpe_type_name,lt_day,base_type,lt_gr_1,lt_gr_2,csi,csi_category,csi_key
79613,NaN,California,Sacramento,0,35,04 35-44,03 21-35,Medium,05 5-10,0,ANDROID,SMARTPHONE,2094,0,08 36+,03 36+,9,happy,1
192069,NaN,California,Los Angeles,0,33,03 25-34,03 21-35,Medium,05 5-10,0,ANDROID,SMARTPHONE,1312,0,08 36+,03 36+,7,neutral,0
417441,NaN,California,San Jose,1,39,04 35-44,04 36-45,Low,04 1-5,1,ANDROID,SMARTPHONE,1196,0,08 36+,03 36+,8,neutral,0
166329,NaN,Oklahoma,Oklahoma City,0,25,03 25-34,03 21-35,High,04 1-5,1,PROPRIETARY,PHONE,282,1,05 7-12,01 до 12,10,happy,1
272636,NaN,Wisconsin,Milwaukee,1,35,04 35-44,03 21-35,Medium,05 5-10,0,ANDROID,SMARTPHONE,4822,0,08 36+,03 36+,7,neutral,0


In [3]:
telecom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   state_type      0 non-null       float64
 1   state           502493 non-null  object 
 2   city            502493 non-null  object 
 3   segment_gender  502493 non-null  int64  
 4   age             502493 non-null  int64  
 5   age_gr_1        502493 non-null  object 
 6   age_gr_2        502493 non-null  object 
 7   arpu_segment    501747 non-null  object 
 8   bucket_tr       502493 non-null  object 
 9   mb_2g_flg       502493 non-null  int64  
 10  os_name         502493 non-null  object 
 11  cpe_type_name   502493 non-null  object 
 12  lt_day          502493 non-null  int64  
 13  base_type       502493 non-null  int64  
 14  lt_gr_1         502493 non-null  object 
 15  lt_gr_2         502493 non-null  object 
 16  csi             502493 non-null  int64  
 17  csi_catego

Как мы видим, на первый взгляд здесь данные почти полные. В столбце "Тип штата" данных вообще нет, но в столбце "Штаты" и "Город" все данные имеются. В открытых источниках попытался найти значение и список типов штата в США, но не нашёл. Почти везде говорится только о самих штатах и города, поэтому можем удалить этот столбец, так как его потеря никак не повлияет на остальные данные и конечный результат. 

Используя функцию ниже, посчитаем процент пропущенных значений для всего датасета и отдельно по каждому столбцу.

In [4]:
def missing_values(data):
    report = data.isna().sum().to_frame()
    report = report.rename(columns = {0: 'missing_values'})
    report['% of total'] = ((report['missing_values'] / data.shape[0]) *100).round(2)
    display(report)
    total = report['% of total'].mean()
    print('Процент пропущенных значений = ', round(total, 3), sep='', end='')

In [5]:
missing_values(telecom_df)

,missing_values,% of total
state_type,502493,100.00
state,0,0.00
city,0,0.00
segment_gender,0,0.00
age,0,0.00
age_gr_1,0,0.00
age_gr_2,0,0.00
arpu_segment,746,0.15
bucket_tr,0,0.00
mb_2g_flg,0,0.00


Процент пропущенных значений = 5.271

С учетом столбца "Тип штата", проценты пропущенных значений составляет почти 5.3%. Надо учитывать, что в этом столбце данные полностью отсутствуют. Давайте теперь удалим эту колонку и посчитаем проценты заново.

In [6]:
telecom_df.drop('state_type', axis=1, inplace=True)

In [7]:
missing_values(telecom_df)

,missing_values,% of total
state,0,0.00
city,0,0.00
segment_gender,0,0.00
age,0,0.00
age_gr_1,0,0.00
age_gr_2,0,0.00
arpu_segment,746,0.15
bucket_tr,0,0.00
mb_2g_flg,0,0.00
os_name,0,0.00


Процент пропущенных значений = 0.008

Теперь процент пропущенных значений 0.008% от всего датасета. Это незначительно и в целом с такими данными можем работать.

Далее посмотрим на наличие дубликатов. Для поиска будем использовать все столбцы, так как здесь анонимные данные пользователей. Если взять только часть столбцов для фильтра, то мы можем получить большое количество дублей, потому что данные могут быть одинаковыми и здесь нет столбца с уникальными данными. 

In [8]:
telecom_df.duplicated().sum()

204

In [9]:
telecom_df.loc[telecom_df.duplicated()].sample(5)

,state,city,segment_gender,age,age_gr_1,age_gr_2,arpu_segment,bucket_tr,mb_2g_flg,os_name,cpe_type_name,lt_day,base_type,lt_gr_1,lt_gr_2,csi,csi_category,csi_key
395442,New York,New York,0,36,04 35-44,04 36-45,Low,04 1-5,0,ANDROID,SMARTPHONE,1760,0,08 36+,03 36+,9,happy,1
301560,Arizona,Phoenix,0,35,04 35-44,03 21-35,Medium,05 5-10,0,ANDROID,SMARTPHONE,477,0,06 13-24,02 13-36,10,happy,1
198758,New York,New York,0,54,05 45-54,06 51-55,New,04 1-5,0,ANDROID,SMARTPHONE,58,1,02 2,01 до 12,10,happy,1
487611,New York,New York,1,40,04 35-44,04 36-45,Medium,06 10-15,0,ANDROID,SMARTPHONE,222,1,05 7-12,01 до 12,10,happy,1
379588,New York,New York,0,48,05 45-54,05 46-50,Medium,04 1-5,0,ANDROID,SMARTPHONE,296,1,05 7-12,01 до 12,10,happy,1


In [10]:
(len(telecom_df.loc[telecom_df.duplicated()] * 100))/len(telecom_df)

0.0004059758046380746

Нашли 204 дублей. Если удалим их, то потеряем 0.0004% от всего датасета и это незначительные потери. Мы вполне можем продолжать работу без дублей и не переживать, что потеряли много данных.

In [11]:
df_dropped_dup = telecom_df.drop_duplicates()

In [12]:
df_dropped_dup.duplicated().sum()

0

Дубли удалили. Теперь детально посмотрим на каждый столбец по отдельности.

In [13]:
df_dropped_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502289 entries, 0 to 502492
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   state           502289 non-null  object
 1   city            502289 non-null  object
 2   segment_gender  502289 non-null  int64 
 3   age             502289 non-null  int64 
 4   age_gr_1        502289 non-null  object
 5   age_gr_2        502289 non-null  object
 6   arpu_segment    501543 non-null  object
 7   bucket_tr       502289 non-null  object
 8   mb_2g_flg       502289 non-null  int64 
 9   os_name         502289 non-null  object
 10  cpe_type_name   502289 non-null  object
 11  lt_day          502289 non-null  int64 
 12  base_type       502289 non-null  int64 
 13  lt_gr_1         502289 non-null  object
 14  lt_gr_2         502289 non-null  object
 15  csi             502289 non-null  int64 
 16  csi_category    502289 non-null  object
 17  csi_key         502289 non-nu

In [14]:
df_dropped_dup['state'].value_counts(normalize=True)

California        0.187416
Texas             0.164553
New York          0.131454
Illinois          0.052942
Arizona           0.051361
Pennsylvania      0.046115
Florida           0.034775
Ohio              0.032471
Tennessee         0.027681
North Carolina    0.024428
Michigan          0.023857
Colorado          0.022137
Indiana           0.019909
Oklahoma          0.016793
Missouri          0.014902
Maryland          0.012614
Massachusetts     0.012566
Wisconsin         0.012347
Washington        0.011256
Columbia          0.010759
Nevada            0.010615
Oregon            0.010337
Kentucky          0.010231
Georgia           0.009905
New Mexico        0.009853
Nebraska          0.009114
Virginia          0.008123
Minnesota         0.006604
Hawaii            0.006534
Kansas            0.005788
Louisiana         0.002560
Name: state, dtype: float64

Больше всего данных о пользователей представлены из штатов California (почти 19%), Texas (чуть больше 16%) и New York (13%), а меньше всего из Louisiana(0,25%) и Kansas(примерно 0,6%).

In [15]:
df_dropped_dup['city'].value_counts().index

Index(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Philadelphia',
       'Phoenix', 'San Antonio', 'Dallas', 'San Diego', 'San Jose', 'Detroit',
       'San Francisco', 'Jacksonville', 'Indianapolis', 'Austin', 'Columbus',
       'Fort Worth', 'Charlotte', 'Memphis', 'Baltimore', 'El Paso', 'Boston',
       'Milwaukee', 'Denver', 'Seattle', 'Nashville', 'Washington',
       'Las Vegas', 'Portland', 'Louisville', 'Oklahoma City', 'Tucson',
       'Atlanta', 'Albuquerque', 'Fresno', 'Sacramento', 'Long Beach', 'Mesa',
       'Kansas City', 'Omaha', 'Cleveland', 'Virginia Beach', 'Miami',
       'Oakland', 'Raleigh', 'Tulsa', 'Minneapolis', 'Colorado Springs',
       'Honolulu', 'Arlington', 'Wichita', 'St. Louis (Saint Louis)', 'Tampa',
       'Santa Ana', 'Anaheim', 'Cincinnati', 'Bakersfield', 'Aurora',
       'New Orleans', 'Pittsburgh', 'Riverside', 'Toledo'],
      dtype='object')

In [16]:
df_dropped_dup['city'].value_counts(normalize=True)

New York        0.131454
Los Angeles     0.058958
Chicago         0.052942
Houston         0.046402
Philadelphia    0.044237
                  ...   
Aurora          0.004097
New Orleans     0.002560
Pittsburgh      0.001877
Riverside       0.001310
Toledo          0.000597
Name: city, Length: 62, dtype: float64

Если посмотреть по городам, то количество данных лидирует у пользователей из New York - 13%. Меньше всего данных из города Toledo - 0,05%.

In [17]:
df_dropped_dup['segment_gender'].value_counts(normalize=True)

 1    0.542188
 0    0.455222
-1    0.002590
Name: segment_gender, dtype: float64

In [18]:
df_dropped_dup['segment_gender'].value_counts()

 1    272335
 0    228653
-1      1301
Name: segment_gender, dtype: int64

Для столбца пол используются только 2 значения, где 1 - женский пол, а 0 - мужской. В данных видим, что есть ещё значение -1. Оно составляет 0,25% от всех данных. Женского пола больше, чем мужского с соотношением 54,2% на 45,5%.
Касательно обозначения -1, можно предположить следующее:
- Во время записи данных опечатались и вместо 1 записали, как -1. В таком случае, можно просто убрать минус и учесть все, как женский пол.
- Были пользователи, которые не могли определить свой пол в не один из предложенных вариантов, из-за чего при записи использовали иное обозначение. 

Точные данные неизвестны, а обратиться к поставщику и запросить объяснение нет возможности, поэтому оставим всё, как есть. Обозначение -1 будет учитывать, как "неизвестный пол".

In [19]:
df_dropped_dup['age'].value_counts(normalize=True).head(15)

33    0.040319
34    0.039860
35    0.038950
36    0.038506
32    0.038096
37    0.037540
31    0.036067
38    0.035499
39    0.034132
30    0.033823
40    0.031850
41    0.030954
29    0.030737
42    0.028934
43    0.027606
Name: age, dtype: float64

Если посмотреть на пользователей по возрастам, то преимущественно пользуются в возрасте от 29 до 41. Теперь посмотрим на всем возврасты пользователей. 

In [20]:
df_dropped_dup['age'].value_counts().index.sort_values()

Int64Index([-1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
            26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
            43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
            60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
            77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89],
           dtype='int64')

Как видим, максимальный возврат пользователей 89 лет, а минимальный 10 лет, если не учитывать аномалиную в -1. Посмотрим на поле, где содержится это значение.

In [21]:
df_dropped_dup[df_dropped_dup['age'] == -1]

,state,city,segment_gender,age,age_gr_1,age_gr_2,arpu_segment,bucket_tr,mb_2g_flg,os_name,cpe_type_name,lt_day,base_type,lt_gr_1,lt_gr_2,csi,csi_category,csi_key
751,Texas,Houston,-1,-1,08 n/a,08 n/a,NaN,04 1-5,0,ANDROID,SMARTPHONE,4055,0,08 36+,03 36+,1,unhappy,-1
1209,Colorado,Denver,-1,-1,08 n/a,08 n/a,NaN,04 1-5,1,ANDROID,SMARTPHONE,4243,0,08 36+,03 36+,5,unhappy,-1
1321,Wisconsin,Milwaukee,-1,-1,08 n/a,08 n/a,NaN,06 10-15,1,ANDROID,SMARTPHONE,2354,0,08 36+,03 36+,1,unhappy,-1
2163,California,San Jose,-1,-1,08 n/a,08 n/a,NaN,04 1-5,0,ANDROID,SMARTPHONE,557,0,06 13-24,02 13-36,3,unhappy,-1
2667,Texas,Houston,-1,-1,08 n/a,08 n/a,NaN,23 95-100,1,IOS,SMARTPHONE,4705,0,08 36+,03 36+,9,happy,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499598,California,Los Angeles,-1,-1,08 n/a,08 n/a,NaN,24 100+,0,ANDROID,SMARTPHONE,520,0,06 13-24,02 13-36,9,happy,1
500834,Texas,Dallas,0,-1,08 n/a,08 n/a,Medium,03 0.1-1,1,ANDROID,SMARTPHONE,220,1,05 7-12,01 до 12,1,unhappy,-1
501246,Oregon,Portland,-1,-1,08 n/a,08 n/a,NaN,04 1-5,0,ANDROID,SMARTPHONE,3550,0,08 36+,03 36+,4,unhappy,-1
502376,New York,New York,0,-1,08 n/a,08 n/a,Medium,04 1-5,0,ANDROID,SMARTPHONE,345,1,05 7-12,01 до 12,5,unhappy,-1


554 строки содержат такое значение. Заметим, что -1 использовался и для столбца "Пол". Всё больше сколняюсь к тому, что -1 использовали для обозначения отсутствующих данных. Посмотрим на другие столбцы. Если походу выявим больше подтверждений гипотезе, то -1 можем заменить на NaN, чтобы данные не искажались.

In [22]:
df_dropped_dup['age_gr_1'].value_counts().sort_values()

08 n/a         554
01 до 16       685
07 66 +      13234
02 16-24     29953
06 55-64     43042
05 45-54     94213
03 25-34    154761
04 35-44    165847
Name: age_gr_1, dtype: int64

In [23]:
df_dropped_dup['age_gr_2'].value_counts().sort_values()

08 n/a         554
01 до 15       685
02 16-20      6925
06 51-55     35871
07 55 +      50206
05 46-50     52041
04 36-45    158654
03 21-35    197353
Name: age_gr_2, dtype: int64

In [24]:
df_dropped_dup[df_dropped_dup['age'] == 17].head(2)

,state,city,segment_gender,age,age_gr_1,age_gr_2,arpu_segment,bucket_tr,mb_2g_flg,os_name,cpe_type_name,lt_day,base_type,lt_gr_1,lt_gr_2,csi,csi_category,csi_key
134,New York,New York,1,17,02 16-24,02 16-20,Low,10 30-35,1,IOS,SMARTPHONE,1652,0,08 36+,03 36+,8,neutral,0
163,Wisconsin,Milwaukee,1,17,02 16-24,02 16-20,Medium,07 15-20,1,ANDROID,SMARTPHONE,807,0,07 25-36,02 13-36,9,happy,1


Сразу рассмотрим два похожих столбца "Сегментация по возрасту". Можем сразу заметить, что использовали две разные сегметации по возрасту. Каждый сегмент пронумерован.
01 для первой группы - это пользователи до 16 лет.
01 для второй группы - пользователи до 15 лет. 
02 в первой группе 16-24 лет, а для второй 16-20 и т.д. 

По итогу мы имеем по 7 возрастных сегментов в каждой группе и 8-ой сегмент для пользователей, чей возврат неизвестен.  

In [25]:
df_dropped_dup['arpu_segment'].value_counts(normalize=True)

Medium    0.515601
High      0.263363
Low       0.187783
New       0.033253
Name: arpu_segment, dtype: float64

Сегмент по ARPU - (ARPU - average revenue per user). Это столбец, где указан средний доход на клиента и он разделен на сегменты. Здесь мы имеем 3 основных сегмента, где больше всего значений: Medium, High, Low, а также отдельно сегмент New - видимо для новых клиентов, по которым ещё неизвестны данные. 

По этим данным можно сделать вывод, что более 51% клиентов входит в средний сегмент по ARPU, то есть они приносят средний доход, а 26% высокий средний доход и около 19% низкий.

Заметка: посмотреть и отсортировать lt_day. Проверить связано количество дней на Сегмент по ARPU.

In [26]:
df_dropped_dup['bucket_tr'].value_counts(normalize=True)

04 1-5         0.224100
05 5-10        0.169046
06 10-15       0.116554
07 15-20       0.086365
08 20-25       0.066420
03 0.1-1       0.055301
09 25-30       0.051247
10 30-35       0.041381
11 35-40       0.032557
12 40-45       0.026057
24 100+        0.022109
13 45-50       0.020835
14 50-55       0.016692
15 55-60       0.013701
16 60-65       0.011237
17 65-70       0.009291
18 70-75       0.007621
19 75-80       0.006578
20 80-85       0.005600
02 0.01-0.1    0.004750
21 85-90       0.004631
22 90-95       0.004199
23 95-100      0.003422
01 0-0.01      0.000207
01 0           0.000100
Name: bucket_tr, dtype: float64

Столбец - "Сегмент по объему скаченного трафика за месяц, в Mb". Здесь мы имеем 24 сегмента по объему скаченного трафика в месяц. Данные указаны в мегабайтах. Судя по этим данным, в среднем в месяц большего всего скачивали от 1 до 15 мб.

In [27]:
df_dropped_dup['mb_2g_flg'].value_counts(normalize=True)

0    0.543944
1    0.456056
Name: mb_2g_flg, dtype: float64

Флаг - наличие трафика в технологии 2G - (Выпадание в 2G - негативное событие для клиента). 0 - флаг отсутсвует. 1 - флаг присутствует и клиент выпадал в тарфик 2G. 54% пользователей не сталкивались с 2G, а вот почти 46% к сожалению, столкнулись. Возможно это окажется сильное влияет на качество удовлетворенности. Позже проверим эту гипотезу. 

In [28]:
df_dropped_dup['os_name'].value_counts(normalize=True)

ANDROID           0.867590
IOS               0.119547
PROPRIETARY       0.004959
OTHER             0.004766
WINDOWS PHONE     0.002582
unknown           0.000265
SYMBIAN OS        0.000205
BADA OS           0.000080
WINDOWS MOBILE    0.000006
Name: os_name, dtype: float64

Преимещуственно пользуются ОС Android - 86,7% пользователей. 11,9% - пользуются IOS. Остальные пользуются другими ОС, частными телефонами, а у 0,02% пользователей данные не известны.

In [29]:
df_dropped_dup['cpe_type_name'].value_counts(normalize=True)

SMARTPHONE                    0.971064
TABLET                        0.018463
PHONE                         0.005039
MOBILE PHONE/FEATURE PHONE    0.004919
ROUTER                        0.000390
USB MODEM                     0.000054
unknown                       0.000036
MODEM                         0.000014
WLAN ROUTER                   0.000006
NETWORK DEVICE                0.000006
HANDHELD                      0.000002
PORTABLE(INCLUDE PDA)         0.000002
MOBILE TEST PLATFORM          0.000002
MIFI ROUTER                   0.000002
Name: cpe_type_name, dtype: float64

Как и следовало ожидать, подавляющее большинство пользуется смартфонами - 97% пользователей. 

In [30]:
df_dropped_dup['lt_day'].value_counts().index.sort_values()

Int64Index([ -21,  -13,  -12,  -11,   -8,   -7,   -6,   -4,   -2,   -1,
            ...
            7895, 8045, 8150, 8190, 8202, 8385, 8490, 8640, 8828, 9162],
           dtype='int64', length=6950)

Стобец "Срок жизни в днях". Сразу можем заметить отрицательные значения, что конечно некорректно. Возможно произошла ошибка/опечатка, либо здесь снова использовали отрицательные значения для пользователей, по которым данные неизвестны. Посмотрим на процент отрицательных значений. 

In [31]:
error_r = [-21,  -13,  -12,  -11,   -8,   -7,   -6,   -4,   -2,   -1]
count = 0
for i in df_dropped_dup['lt_day']:
    if i in error_r:
        count = count + 1
        
count

13

In [32]:
df_dropped_dup[df_dropped_dup['lt_day'] == 0]

,state,city,segment_gender,age,age_gr_1,age_gr_2,arpu_segment,bucket_tr,mb_2g_flg,os_name,cpe_type_name,lt_day,base_type,lt_gr_1,lt_gr_2,csi,csi_category,csi_key
103223,New York,New York,-1,-1,08 n/a,08 n/a,NaN,04 1-5,0,ANDROID,SMARTPHONE,0,1,01 1,01 до 12,5,unhappy,-1
284966,Oregon,Portland,0,-1,08 n/a,08 n/a,NaN,06 10-15,1,ANDROID,SMARTPHONE,0,1,01 1,01 до 12,8,neutral,0
403933,California,Long Beach,1,-1,08 n/a,08 n/a,NaN,05 5-10,0,ANDROID,SMARTPHONE,0,1,01 1,01 до 12,10,happy,1
480641,Oregon,Portland,-1,-1,08 n/a,08 n/a,NaN,07 15-20,0,IOS,SMARTPHONE,0,1,01 1,01 до 12,9,happy,1


In [33]:
(count * 100)/len(df_dropped_dup)

0.002588151442695341

Такие данные составляются 0.002% от всего датасета. В приципе можем их удалить, чтобы они не искажали результаты, так как их объем незначительный, но можно поступить лучше. Скопируем текущий датасет и там удалим все строки, где использовались отрицательные значение, которые нельзя никак интерпретировать. Тогда можем использовать оба датасета для сравнения и анализа разных задач. Сделаем так в конце, когда разберём каждый столбец.

In [34]:
df_dropped_dup['lt_day'].value_counts(normalize=True).sort_values()

7231    0.000002
6645    0.000002
6894    0.000002
7266    0.000002
6901    0.000002
          ...   
37      0.000878
40      0.000886
34      0.000890
57      0.000962
33      0.000987
Name: lt_day, Length: 6950, dtype: float64

In [35]:
df_dropped_dup['base_type'].value_counts(normalize=True)

0    0.831032
1    0.168968
Name: base_type, dtype: float64

В этом столбце указаны 2 вида абонентом: старый (+12 мес) или новый (менее 12 мес) абонент, где 1 - новый и  0 - старый.
83% пользователей старые абоненты, а 16,8% новые.

In [36]:
df_dropped_dup['lt_gr_1'].value_counts(normalize=True)

08 36+      0.545994
06 13-24    0.163219
07 25-36    0.121844
05 7-12     0.081567
04 4-6      0.044673
02 2        0.023928
03 3        0.018728
01 1        0.000046
Name: lt_gr_1, dtype: float64

In [37]:
df_dropped_dup['lt_gr_2'].value_counts(normalize=True)

03 36+      0.545994
02 13-36    0.285063
01 до 12    0.168943
Name: lt_gr_2, dtype: float64

Сегментация по сроку жизни. Также, как и с возвратом, здесь использовали разные сегменты по сроку жизни пользователей в двух группах. В первой группе 9 сегментов, а в второй всего 3. 

In [38]:
df_dropped_dup['csi'].value_counts(normalize=True)

10    0.450709
5     0.101832
8     0.099204
1     0.084408
9     0.074684
7     0.068743
3     0.038342
6     0.032911
4     0.024647
2     0.024520
Name: csi, dtype: float64

Customer satifaction index - индекс удовлетровненности клиента. Заказчик уже сообщил, что CSI вычисляется путем вычета процента несчастных (критиков) из процента счастливых (промоутеров). Вот формула расчета CSI: CSI = (% happy) – (% unhappy).
- Оценка 9-10: Счастливые (Промоутеры): Они любят продукцию вашей компании и, возможно, посоветовали бы ваш бренд потенциальным покупателям. Они всегда совершают повторные покупки и очень ценны для компании.

- Оценка 7-8: Нейтралы: Они не распространяют каких-либо негативных отзывов о бренде, но могут легко перейти к другой компании, если найдут предложение получше. Они не говорят ничего плохого о вас, но ваша продукция не нравится им настолько, чтобы кому-то ее советовать.

- Оценка 0-6: Несчастные (Критики): Они недовольны вашим сервисом и, вероятно, подрывают репутацию вашего бренда своими негативными отзывами.

Видим, что 52,4% счастливые, 16,7% нейтралы и 30,6% несчастные.


In [39]:
df_dropped_dup['csi_category'].value_counts(normalize=True)

happy      0.525393
unhappy    0.306660
neutral    0.167947
Name: csi_category, dtype: float64

Категория CSI. В этом столбце определили категорию CSI согласно оценкам, которые заказчик сообщил нам ранее. Процентное соотношение счастиливых, нейтральных и несчастливых совпадает с тем, что вы посчитали ранее.

In [40]:
df_dropped_dup['csi_key'].value_counts(normalize=True)

 1    0.525393
-1    0.306660
 0    0.167947
Name: csi_key, dtype: float64

В этом столбце указаны ключи для определения категории CSI. 1 - happy, 0 - neutral, -1 - unhappy

## Базовые задачи

### Опишите имеющиеся проблемы в данных

### Проверьте дубликаты, пропуски и выбросы в данных

С датасете нашли следующие проблемы:
1. Полностью утеряны данные в столбце "Тип штата".
2. Почти 5.3% значений пропущены. Однако, если удалить и не учитывать "Тип штата", то пропущенных значений 0.008%.
3. В датасете 204 дубликатов. Это примерно 0.0004% от всего датасета. 
4. В столбце "Пол" есть непонятные нам значения "-1". Они встречаются в 1301 строках.  Заказчик не сообщил для чего они использовали такое обозначение. Есть гипотеза, что для пользователей, чей пол был неизвестен.
5.  В столбце "Возраст" тоже встречается значение -1 в 554 строках. 
6. В столбце "Срок жизни в днях" есть отрицательные значения. Их всего 13. Такие данные составляются 0.002% от всего датасета.

In [41]:
df_dropped_dup.head(3)

,state,city,segment_gender,age,age_gr_1,age_gr_2,arpu_segment,bucket_tr,mb_2g_flg,os_name,cpe_type_name,lt_day,base_type,lt_gr_1,lt_gr_2,csi,csi_category,csi_key
0,Michigan,Detroit,1,45,05 45-54,04 36-45,Medium,04 1-5,1,ANDROID,SMARTPHONE,2320,0,08 36+,03 36+,10,happy,1
1,California,Fresno,0,53,05 45-54,06 51-55,Medium,04 1-5,0,ANDROID,SMARTPHONE,2344,0,08 36+,03 36+,10,happy,1
2,New York,New York,0,57,06 55-64,07 55 +,High,08 20-25,0,ANDROID,SMARTPHONE,467,0,06 13-24,02 13-36,10,happy,1


### Сколько клиентов используют устаревшие операционные системы на своих устройствах?

In [42]:
df_dropped_dup['os_name'].value_counts()

ANDROID           435781
IOS                60047
PROPRIETARY         2491
OTHER               2394
WINDOWS PHONE       1297
unknown              133
SYMBIAN OS           103
BADA OS               40
WINDOWS MOBILE         3
Name: os_name, dtype: int64

В интернете можем найти информацию о современных и устаревших ОС для телефона на текущий момент в мире. 
"Современные операционные системы для мобильных устройств: Android, Kai OS, Lineage OS, Fire OS, Flyme OS, iOS, Sailfish OS, Tizen, Remix OS. Устаревшие, ныне не поддерживаемые программные платформы: Windows 10 Mobile, Symbian, Windows Mobile, Palm OS, webOS, Maemo, MeeGo, LiMo, BlackBerry OS, Firefox OS, Ubuntu Touch, Bada OS." - Ссылка на источник https://ru.wikipedia.org/wiki/Мобильная_операционная_система

Из соверменных ОС, в датасете представлены: ANDROID и IOS. Из устаревших: WINDOWS PHONE, SYMBIAN OS, BADA OS и WINDOWS MOBILE.
Есть также OTHER, unknown и PROPRIETARY - сюда входят другие ОС, которые не были представлены ранее, неизвестные ОС и частные ОС. Остнесём их в категорию - другие.

Теперь посчитаем количество клиентов, которые исплоьзуют современные, устаревшие и другие ОС. 
- Современные: 495828 - 98.71%
- Устаревшие: 1443 - 0.28%
- Другие: 5018 - 0.99%

### Опишите разницу в распределениях значений столбцов age_gr_1 и age_gr_2

In [43]:
df_dropped_dup['age_gr_1'].value_counts().index.sort_values()

Index(['01 до 16', '02 16-24', '03 25-34', '04 35-44', '05 45-54', '06 55-64',
       '07 66 +', '08 n/a'],
      dtype='object')

In [44]:
df_dropped_dup['age_gr_2'].value_counts().index.sort_values()

Index(['01 до 15', '02 16-20', '03 21-35', '04 36-45', '05 46-50', '06 51-55',
       '07 55 +', '08 n/a'],
      dtype='object')

Мы ранее рассматривали эти столбцы и пришли к выводу, что здесь использовали разные сегменты возврастом. Возможно, чтобы понять, клиентов каких возрастов определить в той или иной группе. От этого зависит их покупательская способность. В интернете есть примеры таких определениях таких сегментаций. Если на их основе попытаться разделить текущие сегменты, то получим следующее:
- 1 - 16/ 1 - 15 - дети
- 16 - 24/16 - 20 - молодежь
- 25 - 34/21 - 35 - экономически активное население, самостоятельно принимающее решение о покупке товаров (без детей или с невзрослыми детьми)
- 35 - 44/36 - 45 - экономически активное население (В большинстве своем семейные люди с уже сформированным уровнем достатка и социальным классом. На модель покупательского поведения начинают оказывать влияние дети)
- 45 - 54/46 - 50 - экономически активное население (Их социальный уровень и доход полностью сформированы. Дети данной группы достаточно взрослые и не влияют на модель покупательского поведения. В поведении данной группы начинают проявляться личные интересы и цели, удовлетворения которых ни не смогли добиться в более раннем возрасте)
- 55 - 64/ 51 - 55 - возрослые люди, которые близки к песионному возврасту и уже с взрослыми детьми
- 66 +/55 + -  возрастная группа образует людей пенсионного возраста, не имеющих высокого уровня заработка


### Какой процент штатов США охватывают данные?

In [45]:
df_dropped_dup['state'].value_counts().index

Index(['California', 'Texas', 'New York', 'Illinois', 'Arizona',
       'Pennsylvania', 'Florida', 'Ohio', 'Tennessee', 'North Carolina',
       'Michigan', 'Colorado', 'Indiana', 'Oklahoma', 'Missouri', 'Maryland',
       'Massachusetts', 'Wisconsin', 'Washington', 'Columbia', 'Nevada',
       'Oregon', 'Kentucky', 'Georgia', 'New Mexico', 'Nebraska', 'Virginia',
       'Minnesota', 'Hawaii', 'Kansas', 'Louisiana'],
      dtype='object')

In [46]:
states_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada','New Hampshire',
'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 
'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [47]:
states_in_data = ['California', 'Texas', 'New York', 'Illinois', 'Arizona',
       'Pennsylvania', 'Florida', 'Ohio', 'Tennessee', 'North Carolina',
       'Michigan', 'Colorado', 'Indiana', 'Oklahoma', 'Missouri', 'Maryland',
       'Massachusetts', 'Wisconsin', 'Washington', 'Columbia', 'Nevada',
       'Oregon', 'Kentucky', 'Georgia', 'New Mexico', 'Nebraska', 'Virginia',
       'Minnesota', 'Hawaii', 'Kansas', 'Louisiana']

In [48]:
counter = 0
for i in states_in_data:
    if i in states_list:
        counter = counter + 1
counter        

30

In [49]:
df_dropped_dup['state'].value_counts(normalize=True)

California        0.187416
Texas             0.164553
New York          0.131454
Illinois          0.052942
Arizona           0.051361
Pennsylvania      0.046115
Florida           0.034775
Ohio              0.032471
Tennessee         0.027681
North Carolina    0.024428
Michigan          0.023857
Colorado          0.022137
Indiana           0.019909
Oklahoma          0.016793
Missouri          0.014902
Maryland          0.012614
Massachusetts     0.012566
Wisconsin         0.012347
Washington        0.011256
Columbia          0.010759
Nevada            0.010615
Oregon            0.010337
Kentucky          0.010231
Georgia           0.009905
New Mexico        0.009853
Nebraska          0.009114
Virginia          0.008123
Minnesota         0.006604
Hawaii            0.006534
Kansas            0.005788
Louisiana         0.002560
Name: state, dtype: float64

Данные в датасете охватывают 60% штатов США, то есть 30 штатов из 50. Как мы видели ранее, ,ольше всего данных о пользователей представлены из штатов California (почти 19%), Texas (чуть больше 16%) и New York (13%), а меньше всего из Louisiana(0,25%) и Kansas(примерно 0,6%).